In [1]:
!pip install transformers
!pip install ipywidgets
!pip install IProgress

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import transformers
from transformers import BertTokenizer, BertConfig
from transformers import BertModel, BertForSequenceClassification

In [3]:
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [4]:
model_1 = BertModel(BertConfig.from_pretrained("bert-base-cased"))

In [5]:
model_1

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [6]:
sentence = "hello i have a dog. he is very cute."

In [7]:
tokenized_input = tokenizer(sentence, return_tensors="pt")

In [8]:
tokenized_input

{'input_ids': tensor([[  101, 19082,   178,  1138,   170,  3676,   119,  1119,  1110,  1304,
         10509,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [9]:
output = model_1(**tokenized_input)

In [10]:
output[0]

tensor([[[-0.3602, -1.3389,  1.1931,  ...,  0.2236, -1.7147, -1.3741],
         [ 0.7821,  0.2950,  0.5227,  ...,  0.4816, -0.9162, -2.0736],
         [-0.3923, -0.9394,  0.1356,  ...,  0.4492, -0.8880, -1.0429],
         ...,
         [ 0.4152, -1.4623, -0.6928,  ..., -0.9786, -0.8022, -1.2154],
         [-0.3496,  0.8795,  1.6792,  ..., -0.0402,  0.0855, -1.8874],
         [-1.5621, -0.5006,  0.0178,  ...,  0.0769, -0.4841,  0.3441]]],
       grad_fn=<NativeLayerNormBackward0>)

In [14]:
model_2 = BertModel(BertConfig.from_pretrained("bert-base-cased"))

In [15]:
# model_2.embeddings = None

In [45]:
new_output = model_2(inputs_embeds = output[0], output_hidden_states=True)

In [50]:
for x, y in enumerate(new_output[2]):
    
    print(x, y)

0 tensor([[[-4.2907e-01, -1.4784e+00,  1.3295e+00,  ...,  2.6151e-01,
          -1.9028e+00, -1.5021e+00],
         [ 8.4310e-01,  3.5564e-01,  6.3877e-01,  ...,  5.3007e-01,
          -1.0241e+00, -2.2756e+00],
         [-4.9958e-01, -1.0124e+00,  0.0000e+00,  ...,  5.0764e-01,
          -1.0371e+00, -1.1344e+00],
         ...,
         [ 4.1460e-01, -1.6007e+00, -7.5625e-01,  ..., -1.1261e+00,
          -9.1613e-01, -1.3396e+00],
         [-4.2398e-01,  9.4550e-01,  0.0000e+00,  ..., -6.8783e-04,
           8.9982e-02, -2.0466e+00],
         [-1.8043e+00, -5.1347e-01,  1.2788e-02,  ...,  8.1797e-02,
          -5.6647e-01,  0.0000e+00]]], grad_fn=<NativeDropoutBackward0>)
1 tensor([[[ 0.0263, -0.9214,  0.6803,  ...,  0.4045, -2.0153, -1.4979],
         [ 0.9588,  0.7348,  1.0257,  ...,  0.6752, -0.8185, -2.0210],
         [-0.3627, -0.7736,  0.1394,  ...,  0.6587, -0.8323, -1.1381],
         ...,
         [ 0.6147, -0.3676, -0.4106,  ..., -0.9130, -0.8116, -1.2512],
         [-0.2526,

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [19]:
model_1.config.hidden_size

768

In [13]:
class CustomBERTKuri(nn.Module):

    def __init__(self, model_am, model_ac, model_combined, nr_classes):
        
        super(CustomBERTKuri, self).__init__()
        
        self.model_am = model_am
        self.model_ac = model_ac
        self.model_combined = model_combined
        self.nr_classes = nr_classes
                
        self.fc = nn.Linear(model_combined.config.hidden_size, self.nr_classes)

    def forward(self, tokenized_input):
        
        output = model_am(**tokenized_input)
        
        print(output[0].shape)
        
        output = model_combined(inputs_embeds = output[0])
        
        # output = self.fc(output[0])
    
        return output

In [14]:
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

In [15]:
sentence = "hello i have a dog. he is very cute."

In [16]:
tokenized_input = tokenizer(sentence, return_tensors="pt")

In [17]:
model_am = BertModel(BertConfig.from_pretrained("bert-base-cased"))

In [18]:
model_combined = BertModel(BertConfig.from_pretrained("bert-base-cased"))

In [19]:
custom_model = CustomBERTKuri(model_am, model_am, model_combined, 3)

In [20]:
our_output = custom_model(tokenized_input)

torch.Size([1, 13, 768])


In [12]:
our_output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.3080,  0.2992, -1.3239,  ..., -0.5256,  0.7774, -0.1664],
         [-0.0837,  1.5324, -1.3424,  ..., -1.0329,  2.1938,  0.5698],
         [ 0.6013,  1.8860, -0.0583,  ..., -0.0961,  2.4640,  0.6399],
         ...,
         [ 0.1444,  1.3482, -0.9161,  ..., -0.8573,  0.8058,  1.1466],
         [-0.1071,  1.9615, -0.6014,  ...,  0.0418,  1.4875,  0.3819],
         [-0.7959,  1.1424, -0.2364,  ..., -0.7716,  1.6743,  0.6361]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-3.8542e-01,  5.8907e-01,  4.7898e-01, -4.1342e-01,  3.3790e-01,
         -1.9238e-01, -6.9722e-01,  3.1175e-01,  6.5693e-01,  1.6746e-01,
         -6.3393e-01, -7.6122e-01,  6.4258e-01, -1.1263e-01,  1.0532e-02,
         -3.6647e-01, -2.1205e-01,  2.5337e-01,  6.8979e-01,  1.0673e-01,
          4.5056e-01,  7.6722e-02,  1.0427e-01, -6.5043e-01, -5.8818e-01,
          1.8471e-01,  9.4738e-02,  5.3706e-01,  2.2777e-01, -5.317